In [38]:
import openeo
#import xarray as xr

In [ ]:
def time_series_at_coords(connection, coords, instrument, start_date, end_date, time_chunk = 30):

In [11]:
connection = openeo.connect('earthengine.openeo.org/v1.0').authenticate_oidc(client_id='1029015703668-btbgi4lvp2q35g08fugsauds97sutnfd.apps.googleusercontent.com',
                                                                             client_secret='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')

Authenticated using refresh token.


In [39]:
#connection = openeo.connect('https://openeocloud.vito.be/openeo/1.0.0').authenticate_oidc()
connection = openeo.connect("openeo.dataspace.copernicus.eu/openeo/1.2").authenticate_oidc()

Authenticated using refresh token.


In [3]:
latlonIDs = [[54.3167,10.05,7504.0], [54.4333,9.8167,7521.0], [53.95,10.6833,7539.0]]


In [4]:
def datacube_location_list(coords_list, IDs_list):
    locs_for_datacube =   { "type": "FeatureCollection",
        "features": []
    }
    for coord_index, coords in enumerate(coords_list):
        locs_for_datacube["features"].append({"type": "Feature", "properties": {'name':IDs_list[coord_index]}, "geometry": {"type": "Point", "coordinates": coords[:2]}})
    return locs_for_datacube

In [28]:
def datacube_box(coords, ID, box_width=0.002):
    locs_for_datacube =   { "type": "Polygon",
                           "properties": {'name':ID},
                           "coordinates": [[[coords[1] - box_width, coords[0] - box_width], 
                                           [coords[1] - box_width, coords[0] + box_width], 
                                           [coords[1] + box_width, coords[0] + box_width], 
                                           [coords[1] + box_width, coords[0] - box_width], 
                                           [coords[1] - box_width, coords[0] - box_width]]],
    }
    #for coord_index, coords in enumerate(coords_list):
    #    locs_for_datacube["features"].append({"type": "Feature", "properties": {'name':IDs_list[coord_index]}, "geometry": {"type": "Polygon", "coordinates": [[coords[1] - box_width, coords[0] - box_width], [coords[1] - box_width, coords[0] + box_width], [coords[1] + box_width, coords[0] + box_width], [coords[1] + box_width, coords[0] - box_width]]}})
    return locs_for_datacube

In [18]:
locs = datacube_location_list(latlonIDs, [coords[2] for coords in latlonIDs])

In [19]:
locs

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 7504.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[10.048, 54.314699999999995],
     [10.048, 54.3187],
     [10.052000000000001, 54.3187],
     [10.052000000000001, 54.314699999999995]]}},
  {'type': 'Feature',
   'properties': {'name': 7521.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[9.8147, 54.4313],
     [9.8147, 54.435300000000005],
     [9.818700000000002, 54.435300000000005],
     [9.818700000000002, 54.4313]]}},
  {'type': 'Feature',
   'properties': {'name': 7539.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[10.681299999999998, 53.948],
     [10.681299999999998, 53.952000000000005],
     [10.6853, 53.952000000000005],
     [10.6853, 53.948]]}}]}

In [5]:
latlonIDs

[[54.3167, 10.05, 7504.0], [54.4333, 9.8167, 7521.0], [53.95, 10.6833, 7539.0]]

In [41]:
for location in latlonIDs[:1]:
    #locs_for_datacube = {"type": "Point", "properties": {'name':int(location[2])}, "coordinates": location[:2]}
    locs_for_datacube = datacube_location_list([location], [int(location[2])])
    datacube = connection.load_collection(
        "SENTINEL2_L2A",
        #spatial_extent={"west": 5, "south": 47, "east": 16, "north": 56},
        temporal_extent=["2000-01-01", "2020-12-31"],
        bands=["B04", "B08"]
        )
    #datacube = connection.load_collection(
    #    "COPERNICUS/S2_SR_HARMONIZED",
    #    #spatial_extent={"west": 5, "south": 47, "east": 16, "north": 56},
    #    temporal_extent=["2000-01-01", "2020-12-31"],
    #    bands=["B4", "B8"]
    #)
    box_width = 0.004/2
    locationed = datacube.filter_bbox([location[1] - box_width, location[0] - box_width, location[1] + box_width, location[0] + box_width])
    #locationed = datacube.filter_spatial(geometries=datacube_box(location[:2], int(location[2]), box_width = 0.1))
    res = locationed.save_result(format = "NetCDF", options = {'filename_prefix':f'Station_long_satellites_{int(location[2])}'})#format = "GIF", 
    #                             options = {'filename_prefix':f'Station_long_satellites_{int(location[2])}',
    #                                        'collectionRenderer':'filmstrip'})
    #job = locationed.create_job(title=f'Station_satellites_{int(location[2])}',
    #    description="Sentinel-2 L2A bands",
    #    out_format="NetCDF",
    #    options = {'filename_prefix':f'Station_satellites_{int(location[2])}'}
    #    )
    job = res.execute_batch(title=f'Station_long_satellites_{int(location[2])}')
    jobbed = job.download_results("C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\downloads")
    #job.start()

0:00:00 Job 'j-241212969ae042a6a2bf9f43b9fec117': send 'start'
0:00:16 Job 'j-241212969ae042a6a2bf9f43b9fec117': created (progress 0%)
0:00:21 Job 'j-241212969ae042a6a2bf9f43b9fec117': created (progress 0%)
0:00:28 Job 'j-241212969ae042a6a2bf9f43b9fec117': created (progress 0%)
0:00:36 Job 'j-241212969ae042a6a2bf9f43b9fec117': created (progress 0%)
0:00:46 Job 'j-241212969ae042a6a2bf9f43b9fec117': created (progress 0%)
0:00:58 Job 'j-241212969ae042a6a2bf9f43b9fec117': created (progress 0%)
0:01:13 Job 'j-241212969ae042a6a2bf9f43b9fec117': created (progress 0%)
0:01:33 Job 'j-241212969ae042a6a2bf9f43b9fec117': created (progress 0%)
0:01:57 Job 'j-241212969ae042a6a2bf9f43b9fec117': created (progress 0%)
0:02:27 Job 'j-241212969ae042a6a2bf9f43b9fec117': created (progress 0%)
0:03:04 Job 'j-241212969ae042a6a2bf9f43b9fec117': queued (progress 0%)
0:03:51 Job 'j-241212969ae042a6a2bf9f43b9fec117': queued (progress 0%)
0:04:57 Job 'j-241212969ae042a6a2bf9f43b9fec117': queued (progress 0%)
0:05

C:\Users\wlwc1989\AppData\Local\Temp\ipykernel_9664\3868306853.py:28: UserDeprecationWarning: Call to deprecated method download_results. (Instead use `BatchJob.get_results` and the more flexible download functionality of `JobResults`) -- Deprecated since version 0.4.10.
  jobbed = job.download_results("C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\downloads")
c:\ProgramData\anaconda3\envs\Phenology\Lib\site-packages\openeo\rest\job.py:173: UserDeprecationWarning: Call to deprecated method get_result. (Use `BatchJob.get_results` instead.) -- Deprecated since version 0.4.10.
  return self.get_result().download_files(target)
c:\ProgramData\anaconda3\envs\Phenology\Lib\site-packages\openeo\rest\job.py:177: UserDeprecationWarning: Call to deprecated class _Result. (Use `JobResults` instead) -- Deprecated since version 0.4.10.
  return _Result(self)


In [5]:
for location in latlonIDs:
    #locs_for_datacube = {"type": "Point", "properties": {'name':int(location[2])}, "coordinates": location[:2]}
    locs_for_datacube = datacube_location_list([location], [int(location[2])])
    datacube = connection.load_collection(
        "SENTINEL2_L2A",
        #spatial_extent={"west": 5, "south": 47, "east": 16, "north": 56},
        temporal_extent=["2000-01-01", "2020-12-31"],
        bands=["B04", "B08"]
        )
    locationed = datacube.aggregate_spatial(geometries = locs_for_datacube, reducer = 'mean')
    #res = locationed.save_result(format = "NetCDF", options = {'filename_prefix':f'Station_satellites_{location[2]}'})
    job = locationed.create_job(title=f'Station_long_satellites_{int(location[2])}',
        description="Sentinel-2 L2A bands",
        out_format="NetCDF",
        options = {'filename_prefix':f'Station_long_satellites_{int(location[2])}'}
        )
    #job = res.execute_batch(title=f'Station_satellites_{location[2]}')
    job.start()#get_results().download_files("C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\downloads")

OpenEoApiError: [401] TokenExchangeFailure: Authentication failed. Please login on our marketplace web https://portal.terrascope.be first, and then try authenticate again. (ref: r-241211b6073445f9a115009ec236808e)